# Combine Multiple Embeddings

Combination of embeddings let's you create composite queries that depend on multiple input parameters.

In [1]:
%pip install superlinked==37.1.0

In [2]:
import pandas as pd
from superlinked import framework as sl

pd.set_option("display.max_colwidth", 100)

In [3]:
class Paragraph(sl.Schema):
    id: sl.IdField
    body: sl.String
    like_count: sl.Integer


paragraph = Paragraph()

body_space = sl.TextSimilaritySpace(text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2")
like_space = sl.NumberSpace(number=paragraph.like_count, min_value=0, max_value=100, mode=sl.Mode.MAXIMUM)

## Combining vector spaces

Any number of spaces can be combined in an index.
Indices can be built on top of multiple spaces as simple as that. 

In [4]:
paragraph_index = sl.Index([body_space, like_space])

In [5]:
source: sl.InMemorySource = sl.InMemorySource(paragraph)
executor = sl.InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

In [6]:
source.put(
    [
        {
            "id": "paragraph-1",
            "body": "Glorious animals live in the wilderness.",
            "like_count": 10,
        },
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
            "like_count": 75,
        },
    ]
)

In [7]:
query = sl.Query(paragraph_index).find(paragraph).similar(body_space, sl.Param("query_text")).select_all()

In [8]:
result = app.query(
    query,
    query_text="What makes the AI industry go forward?",
)

In [9]:
sl.PandasConverter.to_pandas(result)

,body,like_count,id,similarity_score
0,Growing computation power enables advancements in AI.,75,paragraph-2,0.565946
1,Glorious animals live in the wilderness.,10,paragraph-1,0.047347
